<a href="https://colab.research.google.com/github/tafartech/Visual-Mastey-Classifier-/blob/main/Visual_Mastey_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Tensorflow Datasets

In [ ]:
!pip install tensorflow tensorflow-datasets

Load CIFAR-10 Dataset

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load CIFAR-10 dataset
(train_data, test_data), info = tfds.load('cifar10', split=['train', 'test'], with_info=True, as_supervised=True)

# Display dataset information
info

*The dataset has been successfully loaded, and the information is as expected. Now, let's proceed with*

# The Exploratory Data Analysis (EDA)

In [ ]:
import matplotlib.pyplot as plt

# Function to display sample images
def show_samples(dataset, num_samples=5):
    plt.figure(figsize=(10, 2))
    for i, (image, label) in enumerate(dataset.take(num_samples)):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(image)
        plt.title(f'Class: {label.numpy()}')
        plt.axis('off')
    plt.show()

# Display sample images from the training set
show_samples(train_data)

*I visualize some sample images from the CIFAR-10 dataset. This will give us a glimpse of the data*

# Data Preprocessing.
In the following code cell, we'll perform data preprocessing, including normalizing pixel values and exploring potential data augmentation:

In [ ]:
# Function to preprocess images
def preprocess_image(image, label):
    # Normalize pixel values to the range [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Apply preprocessing to the datasets
train_data = train_data.map(preprocess_image)
test_data = test_data.map(preprocess_image)

# Display sample preprocessed images
show_samples(train_data)

# Model Building and Training.
we'll construct a deep learning model using TensorFlow for classifying images in the CIFAR-10 dataset.

In [ ]:
from tensorflow.keras import layers, models

# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes in CIFAR-10
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

# Training the Model.
we'll train the model using the preprocessed training data and evaluate its performance on the test set

In [ ]:
# Train the model
history = model.fit(train_data.shuffle(10000).batch(64),
                    epochs=10,
                    validation_data=test_data.batch(64))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data.batch(64))
print(f'Test accuracy: {test_acc}')

# Software Development phase.
we'll create a Python application that allows users to upload images for classification using the trained model.

In [ ]:
from PIL import Image
import numpy as np

# Function to preprocess and classify user-uploaded image
def classify_uploaded_image(file_path):
    # Load and preprocess the image
    img = Image.open(file_path).resize((32, 32))
    img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

    # Make a prediction
    prediction = model.predict(img_array)

    # Display the predicted class
    predicted_class = np.argmax(prediction)
    print(f'Predicted Class: {predicted_class}')

# Example usage (replace 'your_image.jpg' with the actual file path)
classify_uploaded_image('/content/images (1).jpg')

*This code uses the Pillow library to handle image processing. Replace 'your_image.jpg' with the actual file path of the image you want to classify.*

Let's start by fine-tuning the model. We can experiment with different hyperparameters or architectures to see if we can improve performance.

# Model Fine-Tuning
I'll make a small modification to the model architecture, adding more convolutional layers and neurons in the dense layers. Feel free to adjust it further based on your preferences

In [ ]:
# Define a modified model for fine-tuning
fine_tuned_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),  # Additional convolutional layer
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),  # Increased neurons in the dense layer
    layers.Dense(128, activation='relu'),  # Additional dense layer
    layers.Dense(10, activation='softmax')
])

# Compile the modified model
fine_tuned_model.compile(optimizer='adam',
                         loss='sparse_categorical_crossentropy',
                         metrics=['accuracy'])

# Display modified model summary
fine_tuned_model.summary()

let's proceed with training the fine-tuned model. I'll use the same training process as before

In [ ]:
# Train the fine-tuned model
fine_tuned_history = fine_tuned_model.fit(train_data.shuffle(10000).batch(64),
                                          epochs=10,
                                          validation_data=test_data.batch(64))

# Evaluate the fine-tuned model on the test set
test_loss, test_acc = fine_tuned_model.evaluate(test_data.batch(64))
print(f'Test accuracy after fine-tuning: {test_acc}')

# Data Augmentation

In [ ]:
# Introduce data augmentation during training
augmented_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
augmented_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

# Introduce data augmentation
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

# Train the model with data augmentation
augmented_history = augmented_model.fit(
    train_data.shuffle(10000).batch(64).map(lambda x, y: (data_augmentation(x, training=True), y)),
    epochs=10,
    validation_data=test_data.batch(64)
)

# Evaluate the augmented model on the test set
test_loss, test_acc = augmented_model.evaluate(test_data.batch(64))
print(f'Test accuracy with data augmentation: {test_acc}')

# Visualization.
Visualization is crucial for understanding how your model performs during training. Below is an example of how you can visualize the training history and some augmented images

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to plot training history
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

# Plot training history
plot_training_history(history)

# Function to visualize augmented images
def visualize_augmented_images(datagen, image_path):
    # Load and preprocess a sample image
    img = Image.open(image_path).resize((32, 32))
    img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

    # Create a generator with a single sample
    img_generator = datagen.flow(img_array, batch_size=1)

    # Generate augmented images
    plt.figure(figsize=(10, 10))
    for i in range(9):
        augmented_image = img_generator.next()[0]
        plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_image)
        plt.axis('off')
    plt.show()

# Use data generator for visualization
data_generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Visualize augmented images
visualize_augmented_images(data_generator, '/content/images (1).jpg')